In [40]:
from __future__ import print_function,division,absolute_import
from keras.models import model_from_json
from custom_layers import Conv1D_linearphase
import os

In [29]:
checkpoint_name = "/media/taufiq/Data1/heart_sound/models/fold1+compare 2018-05-05 14:39:54.629532/weights.0188-0.8374.hdf5"


In [30]:
model_dir = checkpoint_name[:checkpoint_name.find('fold')]
log_name = checkpoint_name[checkpoint_name.find('fold'):checkpoint_name.find('weights')-1]

In [32]:
if os.path.isdir(model_dir+log_name):
    print("Model directory found")
    if os.path.isfile(os.path.join(model_dir+log_name,"model.json")):
        print("model.json found. Importing")
    else:
        raise ImportError("model.json not found")

Model directory found
model.json found. Importing


In [41]:
with open(model_dir+log_name+"/model.json") as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json,{'Conv1D_linearphase':Conv1D_linearphase})

TypeError: unsupported operand type(s) for %: 'list' and 'int'